# Table of Contents
 <p><div class="lev1"><a href="#Identification-of-dynamic-model-parameters"><span class="toc-item-num">1&nbsp;&nbsp;</span>Identification of dynamic model parameters</a></div><div class="lev2"><a href="#Dynamic-modeling"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Dynamic modeling</a></div><div class="lev2"><a href="#Identification"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Identification</a></div><div class="lev2"><a href="#Parameters"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Parameters</a></div><div class="lev3"><a href="#Standard-and-Base-Parameters"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Standard and Base Parameters</a></div><div class="lev3"><a href="#Essential-Parameters"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>Essential Parameters</a></div><div class="lev3"><a href="#Physical-consistency"><span class="toc-item-num">1.3.3&nbsp;&nbsp;</span>Physical consistency</a></div><div class="lev3"><a href="#Identification-of-difference-between-a-priori-parameters-and-identified-ones"><span class="toc-item-num">1.3.4&nbsp;&nbsp;</span>Identification of difference between a priori parameters and identified ones</a></div><div class="lev3"><a href="#Direct-identification-of-standard-parameters"><span class="toc-item-num">1.3.5&nbsp;&nbsp;</span>Direct identification of standard parameters</a></div><div class="lev2"><a href="#Retrieving-torque-measurements"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Retrieving torque measurements</a></div><div class="lev2"><a href="#Joint-Excitation"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Joint Excitation</a></div><div class="lev2"><a href="#References"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>References</a></div>

# Identification of dynamic model parameters

## Dynamic modeling

__Forward dynamics__:

Given joint torques/forces, find the resulting velocities and accelerations.

$$\begin{equation} \ddot{q} = F(q, \dot{q}, \tau, f_{ext}) \end{equation}$$
with:  
* $F$ - some function
* $q$ - Joint positions in configuration space ($\in \mathbb{R}^n$, generalized coordinates)
* $f_\text{ext}$ - contact forces (at end-effectors)
* $\tau$ - joint torques

The forward dynamics can be solved numerically with the ABA algorithm.

__Inverse dynamics__:

Given the position, velocity and acceleration of joint trajectories, find the necessary
torques/forces of the joints. (Sometimes simply called equations of motion, joint-space form)

$$\begin{equation} \tau = H(q)\ddot{q}+c(q,\dot{q}, f_\text{ext}) \end{equation}$$
with:  
* $H$ - joint-space inertia (or "mass") matrix $\in \mathbb{R}^{6+n_\text{dof} \times 6+n_\text{dof}}$
* $c$ - joint-space bias force (the force which is needed to produce zero acceleration)
  
More detailed, the inverse dynamics with bias forces broken down:  
$$\begin{equation} \tau = H(q)\ddot{q}+C(q,\dot{q})\dot{q} + g(q) + J(q)^T f_\text{ext} \end{equation}$$
with:  
* $C\dot{q}$ - Coriolis and centrifugal forces ($n \times 1$, C $n \times n$)
* $g$ - Gravity terms ($n \times 1$)
* $J$ - Jacobian of the end-effector(s)
* $f_\text{ext}$ - Wrench excerted by external environment on link
* (friction is missing, for that see p.228 Khalil)

The inverse dynamics problem can be solved with the RNEA algorithm (which can also give $c(q,\dot{q}, f_\text{ext})$).

The inverse dynamics can also be written in regressor form:

$$\begin{equation} \tau = Y(\theta, \dot{\theta}, \ddot{\theta}) x + \rho \end{equation} $$
with:  
* $Y$ - dynamics regressor ($n_\text{dof} \times 10n_\text{links}$) (Jacobian of $\tau$ with respect to vector $x$)
* $x$ - inertial parameter vector ($10n_\text{links} \times 1$)
* $\rho$ - errors due to measurement noise and modelling errors ($n_\text{dof} \times 1$)

The inertia parameters usually consist of mass in kg (1 param), location of center of mass relative to joint (3, x,y,z) and the rotational inertia in each axis around the COM (6, xy, xx, xz, yy, yz, zz), so 10 parameters for each link. Additionaly, e.g. 4 friction parameters are possible (total inertia of moment for rotor and gears, viscous and coulomb coefficients and an offset parameter), (Gautier, 2013, p.1430).
Inverting eq. (4) for $x$ (further down) assumes that the equations are linear in the unknown parameters. This is however only true for Recursive Newton-Euler if the parameters are given in a modified form. The inertia values/tensors need to be given in relation to the link frame (URDF uses relative to the COM). The COM needs to be given as first moment of mass, so coordinates times mass. See also Khosla, 1985.

In CoDyCo/iDynTree notation (https://github.com/robotology/idyntree/blob/master/doc/dcTutorialCpp.md):  
"For floating-base dynamics, the dynamics regressor is a $(6+n_\text{dofs})\times (10 \cdot n_\text{dofs})$ matrix $Y$ such that: $Y \pi = M(q) \frac{d(v)}{dt} + C(q,v)v + g(q)$ with $M(q)$, $C(q,v)$ and $g(q)$ defined in http://wiki.icub.org/codyco/dox/html/dynamics_notation.html. The $\pi$ vector is a $10 \cdot n_\text{dofs}$ inertial parameters vector, such that the elements of the vector from the $((i-1) \cdot 10)$-th to the $((i \cdot 10) - 1))$-th belong to the i-th link. For more details on the inertial parameters vector, check https://hal.archives-ouvertes.fr/hal-01137215/document".


## Identification

We're looking to find the (standard) parameter vector $x$ (mass, com, inertia for each joint) in the inverse dynamics regressor equation.

iDynTree can calculate the dynamics regressor $Y$ numerically (using RNEA) for each state, which is dependent on the joint-link tree and on the system state ($q, \dot{q}, \ddot{q}$). For Walk-Man, the contact forces with the ground at the feet can be measured. For fixed base dynamics, the only force is coming through the constrained base link (for which the Jacobian is $J = [I \quad 0]$ and the contact forces vanish).
Robotran or openSYMORO gives the regressor as symbolic equations that are dependent on the system state and the dynamical and some structural parameters.

When torques and/or contact forces are known and Y is retrieved through iDynTree (also $J^T$ of each contact link), we can calculate 

$$\begin{equation} Yx = \tau - \sum_{l \in L} J_{l}^T f_l^{ext} \end{equation}$$
$$\begin{equation} x = Y^{-1}\tau - Y^{-1}\sum_{l \in L} J_{l}^T f_l^{ext} \end{equation}$$
(using pseudoinverse for the general non-invertible case)
$$\begin{equation} \tilde{x} = Y^{+}\tau - Y^{+}\sum_{l \in L} J_{l}^T f_l^{ext} \end{equation}$$

(Y is only dependent on motion data and model, and can be calculated without knowing x)

In order to do identification of $x$ without having to know the contact forces, the HyQ identification code uses a null-space multiplication to eliminate the contact forces (they don't have sensors in the feet).
 
$$\begin{equation} Yx = \tau - J^T f_\text{ext} \end{equation}$$
$$\begin{equation} N(J^T)Yx = N(J^T)\tau - \underbrace{N(J^T)J^T f_\text{ext}}_{=0} \end{equation}$$
$$\begin{equation} x = [N(J^T)Y]^{-1}N(J^T)\tau \end{equation}$$

If only contact forces are known, there is also the possibility to just look at the upper equations of the matrix formulation and leave out the joint torques (see: Ayusawa, Venture: "Identifiability and identification of inertial parameters using the underactuated base-link dynamics for legged multibody systems").

## Parameters

### Standard and Base Parameters
The parameters $x$ are called the standard parameters as they occur in the normal dynamics equations. The rank of the regressor is usually smaller than its number of columns, i.e. some columns are linearly dependent (matrix is rank deficient). There are therefore infinite solutions for the equations of which only some are physically consistent and meaningful (see below) because the matrix is singular.
Before inverting the equation to retrieve the minimum identifiable parameters (base or identifiable parameters), we need to reduce the regressor $Y$'s columns to the linearly independent one, using an algorithm given by Gautier, 1990, using SVD or QR or using closed form rules (Khalil). That way, some base parameters that are not significant are removed and some will be linear combinations of two or more standard parameters, that were linearly dependent. The base parameters are linearly independent but the choice is not unique (there are multiple possibilities of combining the columns).

The regressor for the base parameters can be retrieved from the SVD, where the first $\text{rank}(Y_\text{std})$ columns of V are the basis vectors of the identifiable subspace. There is computeFloatingBaseIdentifiableSubspace() in iDynTree which calculates this orthogonal basis $B$ of the identifiable parameters subspace. It can be used to project from the standard regressor to the base regressor, i.e. $Y_\text{base} = Y_\text{std}B$ (See Traversaro, 2015, p.3). Other methods include the QR decomposition or the SVD with a permutation matrix that explicitly defines which columns are changed and combined (See Gautier, 1990).

The ordinary least squares can be improved by weighting with the inverse of the standard deviation of the error calculated with the ordinary LS (Gautier, 2013, p.1431).
That way, parameters with smaller errors deviation are given higher priority.

After obtaining the base parameters $\tilde{x}$, they can be projected back to the standard basis in order to use them for e.g. simulation models or modifying the URDF file. When using a symbolic regressor, both can stay in the base subspace for e.g. torque prediction.
This way, the parameters are however not necessarily physically meaningful (see physical consistency).

Using the transpose of $B$ (here equal to inverse), we can calculate $x_\text{std} = B^T x_\text{base}$. This way, a mean estimation is produced to get the unknown distribution of the linear combinations back to multiple single parameters.

### Essential Parameters

The base parameters might still have elements that are (get identified as?) almost zero and therefore don't contribute much to the dynamics calculations. Possibly because they have not been excited well in the experiment that was measured. They can therefore be cancelled (set to zero or params/regressor columns deleted) to retrieve a smaller set of parameters called the essential parameters in order to simplify the model further. The estimation results will not exactly be the same as the model will be expressed with fewer parameters and equations.

The essential base parameters can be determined using the relative standard deviation of the identifed parameter vector, using standard deviation of the error of the estimated torques (See Gautier, 2013). Iteratively, the parameter with the largest relative std dev will be cancelled and the estimation repeated. This is done until a ratio of min and max relative std dev on the parameters is reached that is between 30 and 20. That way, the similarity of estimation "certainty" of the remaining parameters is increased. The error itself is not looked at.  Other criteria for stopping the model reduction because of error are the F-statistic (Janot 2014) or the lack-of-fit error (based on mean least squares) but these measures assume that the error is normal distributed which is not always the case. Janot recommends improving measurements, filtering etc. (feasible for us?). Also, it is unclear (to me) how these measures are related to amount of samples selected, they seem to change a lot on the same data but e.g. every other sample. 
Alternatively, looking at the error of the estimation in relation to the maximal torques range of each joint or the maximum measured torque ranges allow stopping before the prediction error gets too large (torque prediction should ultimately still be good, even if the parameters are not estimated close to the actual values).

The torque error $\rho$ in $(4)$ includes measurement error, error due to unmodelled effects (linear and non-linear) and the error from model reduction. Before reduction, only the modelling error $\rho_j$ of sample $j$ can be determined (the real torques are not known) as the mean of the difference of measured torques $\tau_i$ and the corresponding estimated torques $\hat{\tau_i}$ of joint $i$:

$$ \rho_j =\frac{1}{n_\text{dofs}} \sum_{i=0}^{n_\text{dofs}}(\tau_i - \hat{\tau_i})^2 $$

and $\rho$ being the vector of the errors for all measured samples $j$, $\rho = [\rho_0, \rho_1, \dots \rho_N]^T$

The percentual error in regard to e.g. the maximum torque (range) is:
$$ mse / mean(torqranges) $$

When using the QR decomposition or permutation matrices for determining which std parameters are regrouped into the base parameters, it is known which of the standard parameters belong to each of the base parameters. Having determined the indices of the essential parameters within the base parameters, and knowing the indices of the base parameters within the standard parameters, the essential parameters within the standard parameters can also be determined.
Each base parameter $b$ can correspond to a linear combination of $n$ standard parameters $a_n$ $k_1a_1+k_2a_2+...+k_na_n$. Each of the $a_n$ is assumed to be essential if $b$ is essential (to be inclusive; some of them can have no influence).


### Physical consistency

Standard parameters are called physical consistent if masses are positive, the inertia matrix is positive definite and the center of mass is located inside of the segment. The moments of inertia need to respect the triangle inequality.
The base (and base essential parameters) are usually not consistent, because they are only derived to minimize the error norm of the torque prediction.
There are multiple ways of forcing the parameters to be consistent, often with expensive calculations.

### Identification of difference between a priori parameters and identified ones

Using parameters from the CAD model, it is possible to determine physical consistent standard parameters that are close to the a priori knowledge (Gautier, 2013, p.1432) and are still optimal with regard to error norms (other methods may be worse than before).
Let $x^\text{ref}_\text{std}$ be the a priori CAD parameter vector and $\tau^\text{ref}$ be the torques estimated with these parameters by $\tau^\text{ref} = Y_\text{std}x^\text{ref}_\text{std}$.
Subtracting the regressor equation of the identification problem from the estimated parameters, we get:
$$\tau - \tau^\text{ref} = Y_\text{std} (x_\text{std} - x^\text{ref}_\text{std}) + \rho $$
$$ \iff \Delta \tau = Y_\text{std} \Delta x_\text{std} + \rho $$

It is shown that additionally using the information from the reference parameters about which parameters are essential and weighing the regressor and parameter vectors with it, the optimal consistent parameters are found.

### Direct identification of standard parameters

(Gautier, 2013)
It is possible to modify the regressor to remove the singular values and then identify the standard parameters directly. The non-singular matrix $\hat{Y}_\text{std}$ closest to $Y_\text{std}$ with respect to the Frobenius norm is $$\hat{Y}_\text{std} = Y_\text{std} - U_{n_b:n_\text{st}}\Sigma_{n_b:n_\text{st}}{V_{n_b:n_\text{st}}}^T = Y_\text{std} - \sum\limits_{k=n_b+1}^{n_{std}}{s_k {U_k V_k}^T}$$
with $U \Sigma V^T = \text{SVD}(Y_\text{std})$ and with $s_k$ the $k$-th value on the diagonal of $\Sigma$ and $U_k$ and $V_k$ being the $k$-th column of $U$ and $V$.
The identification with the pseudoinverse can then be done as before with the base parameters, now identifying the standard parameters directly.


## Retrieving torque measurements

The torque measurements should measure the deflection between link $i$ and child link $i+1$. For the position, velocity and acceleration, it is likely necessary to differentiate the velocity and possibly also the position depending on what measurements are available. Since differentiation has the effect of high-pass filtering and therefore adding new noise, the data can be low-pass filtered (e.g. with an appropriately designed butterworth filter). Even with butterworth, this can introduce phase-shift. Another option is to assume a curve shape model,
fit it to a few data points and differentiate it analytically, it is also possible to use simple median filtering or RANSAC for additional elimation of outliers.

It is often done in previous work to use multiple experimental setups with separate body parts (going up the chain) while fixating the rest of the body to improve the estimation accuracy. For the iCub, the arms were "cut" at the position of the 6 DOF F/T sensors so that the measurement gives the contact forces for a fixed base scenario. The limbs should move with a standardized trajectory excitation that highlights all the influencing columns and has sufficient velocities, accelerations and forces appearing. The dynamic equations don't include friction or other non-linear effects. We can get the Jacobian(s) from iDynTree.

Using $N$ measurement samples at time instants $t_1 \dots t_N$, the regressor matrix is extended by v-stacking regressors for each measurement and the same for the torque vectors.

$$\begin{bmatrix} 
   Y(t_1)\\ 
   \vdots \\
   Y(t_N)\\
 \end{bmatrix}x = \begin{bmatrix} 
   \tau(t_1)\\ 
   \vdots \\
   \tau(t_N)\\
 \end{bmatrix} - \begin{bmatrix} 
   {\sum_{l \in L} J_{l}^T f_l^{ext}}^1\\ 
   \vdots \\
   {\sum_{l \in L} J_{l}^T f_l^{ext}}^N\\
 \end{bmatrix} $$
$$ x = \begin{bmatrix} 
   Y(t_1)\\ 
   \vdots \\
   Y(t_N)\\
 \end{bmatrix}^{+}\begin{bmatrix} 
   \tau(t_1)\\ 
   \vdots \\
   \tau(t_N)\\
 \end{bmatrix} - \begin{bmatrix} 
   Y(t_1)\\ 
   \vdots \\
   Y(t_N)\\
 \end{bmatrix}^{+}\begin{bmatrix} 
   (\sum_{l \in L} J_{l}^T f_l^{ext})^1\\ 
   \vdots \\
   (\sum_{l \in L} J_{l}^T f_l^{ext})^N\\
 \end{bmatrix} $$
 
 The resulting matrices should have the following dimensions ($n_\text{dof}$ - Degrees of Freedom, $n_\text{links}$ - Nr. of Links):

 $Y_1^N$: $N \cdot n_\text{dof}\times (10\cdot n_\text{links})$
 
 $\tau^N_1$: $N \cdot n_\text{dof}$
 
 The equations only preduce sensible results if there are sufficient measurements and the measurements are reasonably noise free. Otherwise, a maximum-likehood estimator could be a good option which estimates the noise with known constant variance. Swevers describes one way of using an MLE and that it is robust against non-linear effects in the model such as modeled friction.

## Joint Excitation

A good estimation should yield physically consistent parameters and only produce small prediction errors (measured torques and simulated torques) with those estimated parameters.
Siciliano recommends sufficiently rich trajectories that should however not excert unmodelled dynamic effects like elasticity.

In order to get meaningful sample data that allows good estimation, trajectories can be generated by optimization of e.g. the prediction error or the condition number of the regressor matrix.
Swevers, Gensemann (1997) use Fourier series on all joints to generate periodic excitation and use a common pulsation (i.e. frequency) so the excitation as a whole also stays periodic (multiples of the base frequency?). This way it is said to be easier to get time series measurements because averaging can be done to reduce noise, noise can be estimated and velocities and accelerations can be calculated analytically using the FFT to get near noise-free values. The joint angles for the $i$-th of $n$ joints are given by $N$ coefficients:

$$ q_i(t) = \sum_{l=1}^{N_i} \frac{a_l^i}{\omega_f l} \text{sin}(\omega_f lt)- \frac{b_l^i}{\omega_f l} \text{cos}(\omega_f lt) + q_{i0}$$

It is therefore parameterized by the Fourier coefficients $a_l^i$ (amplitude of sin), $b_l^i$ (amplitude of cos) and $q_i0$ (offset on the position trajectory) for each term $l$ of each joint $i$.

Finding the right parameters is then an optimization problem to get ideal trajectories.
The "only correct" criterion for experimentation design is the covariance matrix of the estimated model parameters which when using their maximum-likelyhood estimation is equal to $(F^t \Sigma^{-1}F)^{-1}$ with $F$ the condition number. Optimization takes the model constraints into account, too.

Ting, et. al (2006) describe a method based on Bayesian estimation that automatically detects noise and in a post-processing step, projects the estimate within consistency constraints (positive inertia values and parallel axis theorem). Higher measurement accuracy is claimed.

Ding, et. al. (2015) use an "artificial bee" algorithm to find the parameters for the fourier series parameters.

All methods optimize on minimum error of the identification trajectories, so parameters predict trajectories best that are similar or equal to those trajectories because only some links are exciting some of the parameters. Using the essential parameter approach, this results in only the most likely parameters being changed from the a priori values that are relevant for this task.

It may become beneficiary to use specific optimized excitation trajectory experiments initially to have get a well identified general model but nevertheless, have task specific identification during the robot lifetime.

Instead of generating trajectories, samples from normal movement data of existing behaviors can be selected. The criterion for adding new data to the existing dataset or not is still to be determined but could be similar to existing optimization cirteria.
Venture, et. al (2009) select data from an existing dataset to identify and calculate the condition number for sub-matrices of the regressor (select certain columns with all rows) to get information about which links are excerted.
The final regressor is put together in such a way that all columns have low values. 

## References

(Notation mostly from here http://www.scholarpedia.org/article/Robot_dynamics)

Swevers, Gansemann et. al., 1997: Optimal Robot Excitation and Identification

Ting, Mistry, Peters, et. al., 2006: A Bayesian Approach to Nonlinear Parameter Identification for Rigid Body Dynamics

Ding, Wu, et. al., 2015: Dynamic Model Identification for 6-DOF Industrial Robots, http://www.hindawi.com/journals/jr/2015/471478/

Khosla, 1985: Parameter Identification of Robot Dynamics

Siciliano, Scaviccio, et. al., 2009: Robotics Modelling, Planning and Control, Springer

Traversaro, Del Prete, et. al., 2015: Inertial parameters identification and joint torques estimation with proximal force/torque sensing

Gautier, 2013: Identification of Consistent Standard Dynamic Parameters of Industrial Robots

Gautier, 1990: Numerical Calculation of the base Inertial Parameters of Robots